<a href="https://colab.research.google.com/github/Satish-970/ATS_Checker_Job-Description/blob/main/ats_matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ATS Resume Matcher: Your Guide to a Better Resume

This tool helps you evaluate how well your resume matches a job description, providing an ATS score and suggestions for improvement. It's designed to help you pass through Applicant Tracking Systems (ATS) and get your resume seen by recruiters.

To use it, run all the code cells sequentially. You'll be prompted to install dependencies, then access a web interface to upload your resume (PDF or DOCX) and paste the job description. No Google Drive sign-in is required - uploads are done directly from your device via a Streamlit web app. 🚀

### How to Run
1. Run the first cell to install dependencies.
2. Run the second cell to start the Streamlit app.
3. Open the provided local URL (e.g., `http://localhost:8501`) in your browser.
4. Upload your resume and paste the job description to get results.

**Note**: If running locally, ensure you have Python installed. For deployment, see instructions at the end.

In [ ]:
# Install necessary libraries
!pip install streamlit spacy PyPDF2 python-docx scikit-learn matplotlib --quiet
!python -m spacy download en_core_web_sm --quiet

print("Setup complete! ✅")

In [ ]:
# Define the Streamlit app

%%writefile ats_matcher.py
import streamlit as st
import spacy
import PyPDF2
from docx import Document
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import os

# Streamlit app title and description
st.title("ATS Resume Matcher")
st.markdown("""
This tool evaluates how well your resume matches a job description, providing an ATS score and suggestions for improvement. 
Upload your resume (PDF or DOCX) and paste the job description to get instant results! 🚀  
**No sign-in required** - upload directly from your device.
""")

# Load spaCy model
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    st.info("Downloading spaCy model...")
    os.system("python -m spacy download en_core_web_sm --quiet")
    nlp = spacy.load('en_core_web_sm')

# Function to extract text from PDF
def extract_pdf_text(pdf_file):
    text = ''
    try:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            extracted_page_text = page.extract_text()
            if extracted_page_text:
                text += extracted_page_text + ' '
    except Exception as e:
        st.error(f"Error extracting text from PDF: {e}")
        return ""
    return text.strip()

# Function to extract text from DOCX
def extract_docx_text(docx_file):
    text = ''
    try:
        doc = Document(docx_file)
        for paragraph in doc.paragraphs:
            text += paragraph.text + ' '
    except Exception as e:
        st.error(f"Error extracting text from DOCX: {e}")
        return ""
    return text.strip()

# Function to extract text from uploaded resume
def extract_resume_text(uploaded_file):
    file_extension = uploaded_file.name.lower().split('.')[-1]
    if file_extension == 'pdf':
        return extract_pdf_text(uploaded_file)
    elif file_extension in ['docx', 'doc']:
        return extract_docx_text(uploaded_file)
    else:
        st.error("Unsupported file format. Please upload PDF or DOCX files only. ⚠️")
        return ""

# Function to extract keywords
def extract_keywords(text):
    doc = nlp(text.lower())
    keywords = set()
    
    # Comprehensive list of technical and professional skills
    tech_skills = {
        'python', 'java', 'javascript', 'sql', 'html', 'css', 'react', 'angular', 
        'vue', 'node.js', 'django', 'flask', 'spring', 'hibernate', 'mongodb', 
        'postgresql', 'mysql', 'oracle', 'aws', 'azure', 'gcp', 'docker', 
        'kubernetes', 'jenkins', 'git', 'github', 'gitlab', 'agile', 'scrum', 
        'kanban', 'jira', 'confluence', 'tensorflow', 'pytorch', 'keras', 
        'pandas', 'numpy', 'scipy', 'matplotlib', 'seaborn', 'tableau', 
        'powerbi', 'excel', 'r', 'scala', 'spark', 'hadoop', 'kafka', 
        'elasticsearch', 'redis', 'graphql', 'rest', 'api', 'microservices',
        'devops', 'ci/cd', 'linux', 'windows', 'macos', 'bash', 'powershell',
        'communication', 'leadership', 'teamwork', 'management', 'analytics',
        'problem-solving', 'debugging', 'testing', 'automation'
    }
    
    # Extract nouns, proper nouns, and technical skills
    for token in doc:
        if token.pos_ in ['NOUN', 'PROPN'] and len(token.text) > 2 and token.text.isalpha():
            keywords.add(token.text)
        elif token.text in tech_skills:
            keywords.add(token.text)
    
    # Handle multi-word skills and phrases
    text_lower = text.lower()
    multi_word_skills = [
        'machine learning', 'data analysis', 'data science', 'project management',
        'artificial intelligence', 'deep learning', 'natural language processing',
        'computer vision', 'big data', 'business intelligence', 'data mining',
        'software development', 'web development', 'mobile development',
        'full stack', 'front end', 'back end', 'database management',
        'cloud computing', 'cyber security', 'network security',
        'quality assurance', 'user experience', 'user interface'
    ]
    
    for skill in multi_word_skills:
        if skill in text_lower:
            keywords.add(skill)
    
    return keywords

# Function to calculate ATS score
def calculate_ats_score(resume_text, job_description):
    documents = [resume_text, job_description]
    vectorizer = TfidfVectorizer(
        stop_words='english', 
        ngram_range=(1, 2),
        max_features=1000,
        lowercase=True
    )
    tfidf_matrix = vectorizer.fit_transform(documents)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return similarity * 100

# Function to provide improvement suggestions
def get_suggestions(resume_keywords, job_keywords):
    missing = job_keywords - resume_keywords
    if missing:
        missing_list = sorted(list(missing))[:10]  # Top 10 suggestions
        return f"Consider adding these keywords to your resume: {', '.join(missing_list)} ✍️"
    return "Excellent! Your resume contains all the key skills from the job description. 🌟"

# Function to provide score interpretation
def get_score_feedback(score):
    if score >= 80:
        return "Excellent match! Your resume aligns very well with this job. 👍"
    elif score >= 65:
        return "Good match! Minor improvements could boost your score. 😊"
    elif score >= 50:
        return "Moderate match. Review suggestions to improve alignment. 🤔"
    elif score >= 35:
        return "Below average match. Consider significant improvements. 😟"
    else:
        return "Low match. Major revisions recommended to align with job requirements. 😥"

# Function to visualize the score
def visualize_score(score):
    # Color coding based on score ranges
    if score >= 80:
        color = 'green'
        performance = 'Excellent'
    elif score >= 65:
        color = 'lightgreen'
        performance = 'Good'
    elif score >= 50:
        color = 'orange'
        performance = 'Moderate'
    elif score >= 35:
        color = 'red'
        performance = 'Below Average'
    else:
        color = 'darkred'
        performance = 'Low'
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(['ATS Match Score'], [score], color=color, alpha=0.8, width=0.6)
    
    ax.set_ylim(0, 100)
    ax.set_ylabel('Match Percentage (%)', fontsize=12)
    ax.set_title('Resume ATS Match Analysis', fontsize=14, fontweight='bold')
    
    # Add score text on the bar
    ax.text(0, score + 2, f'{score:.1f}%\n({performance})', 
            ha='center', va='bottom', fontsize=14, fontweight='bold')
    
    # Add reference lines
    ax.axhline(y=80, color='green', linestyle='--', alpha=0.3, label='Excellent (80%+)')
    ax.axhline(y=65, color='lightgreen', linestyle='--', alpha=0.3, label='Good (65%+)')
    ax.axhline(y=50, color='orange', linestyle='--', alpha=0.3, label='Moderate (50%+)')
    ax.axhline(y=35, color='red', linestyle='--', alpha=0.3, label='Below Average (35%+)')
    
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    st.pyplot(fig)

# Main Streamlit app logic
def run_ats_checker():
    st.subheader("Step 1: Upload Your Resume")
    uploaded_file = st.file_uploader("Choose a resume file (PDF or DOCX)", type=['pdf', 'docx', 'doc'])
    
    if uploaded_file:
        st.success(f"Successfully uploaded: {uploaded_file.name} 🎉")
        resume_text = extract_resume_text(uploaded_file)
        if not resume_text or len(resume_text.strip()) < 100:
            st.warning("Very short resume text extracted. Please ensure your PDF/DOCX is readable. 🧐")
            return
        
        st.subheader("Step 2: Paste the Job Description")
        job_desc = st.text_area("Paste the complete job description here:", height=200)
        
        if job_desc and len(job_desc.strip()) < 50:
            st.warning("Job description seems very short. Consider pasting the complete job posting for better analysis. 🤏")
        
        if st.button("Analyze Resume"):
            if not job_desc.strip():
                st.error("Job description cannot be empty. Please paste a valid job description. 🚫")
                return
            
            st.info("Analyzing your resume against the job description... 📊")
            
            try:
                # Extract keywords and calculate score
                resume_keywords = extract_keywords(resume_text)
                job_keywords = extract_keywords(job_desc)
                ats_score = calculate_ats_score(resume_text, job_desc)
                
                # Display results
                st.markdown("### ATS Match Analysis Results")
                st.markdown(f"**ATS Match Score:** {ats_score:.2f}%")
                st.markdown(f"**Score Category:** {get_score_feedback(ats_score)}")
                
                st.markdown("#### Keyword Analysis")
                st.markdown(f"- Resume keywords found: {len(resume_keywords)}")
                st.markdown(f"- Job description keywords: {len(job_keywords)}")
                st.markdown(f"- Matching keywords: {len(resume_keywords & job_keywords)}")
                
                matching_keywords = resume_keywords & job_keywords
                if matching_keywords:
                    st.markdown(f"**Matching skills found:** {', '.join(sorted(list(matching_keywords))[:15])} ✅")
                
                st.markdown("#### Improvement Suggestions")
                st.markdown(get_suggestions(resume_keywords, job_keywords))
                
                # Show visualization
                st.markdown("#### ATS Score Visualization")
                visualize_score(ats_score)
                
                # Additional tips
                st.markdown("#### Additional Tips to Boost Your Score ✨")
                if ats_score < 65:
                    st.markdown("""
                    - Use exact keywords from the job description in your resume.
                    - Include relevant skills in a dedicated skills section.
                    - Tailor your experience descriptions to match job requirements.
                    - Use industry-standard terminology.
                    """)
                st.markdown("""
                - Ensure your resume is in a simple, ATS-friendly format.
                - Use standard section headings (Experience, Skills, Education).
                - Avoid images, tables, and complex formatting.
                """)
                
                st.markdown("**Analysis Complete! Good luck! 🍀**")
            
            except Exception as e:
                st.error(f"Error during analysis: {e}")
    
    else:
        st.info("Please upload a resume to start the analysis.")

# Run the app
if __name__ == "__main__":
    run_ats_checker()

print("Streamlit app code saved to 'ats_matcher.py'. Run the next cell to start the app. ⚙️")

In [ ]:
# Run the Streamlit app
# This will start a local web server. Open the provided URL (e.g., http://localhost:8501) in your browser.
!streamlit run ats_matcher.py

## Deployment Instructions

To share this app online (e.g., via Streamlit Cloud):
1. **Push to GitHub**:
   - Save this notebook as `ats_matcher.ipynb`.
   - Create a GitHub repository and upload the notebook.
   - Create a `requirements.txt` file with the following content:
     ```
     streamlit
     spacy
     PyPDF2
     python-docx
     scikit-learn
     matplotlib
     ```
2. **Deploy to Streamlit Cloud**:
   - Sign up at `cloud.streamlit.io`.
   - Connect your GitHub repository.
   - Select `ats_matcher.py` as the main script and deploy.
3. **Access the App**:
   - Once deployed, you'll get a public URL where users can upload their resume and job description.

**Note**: Ensure your resume files are text-based (not scanned images) for accurate text extraction. For local testing, run the previous cell and access the app in your browser.